https://avielrs.github.io/Intro-Market-Basket-Analysis/

https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store

https://github.com/nurlailiis/Association-Algorithm/blob/master/Dataset/control_data.csv

https://github.com/krishna-aditi/product-recommendation-based-on-market-basket-analysis-and-rfm-analysis

https://github.com/huytjuh/Recommender-System-Basket-Analysis

https://github.com/Sriram-infohub/Market-Basket-Analysis-and-Recommendation-System/blob/main/AdvML_4(Retail).ipynb

https://github.com/AjNavneet/Rule-Based-Recommendation-Market-Basket-Analysis-Retail

https://github.com/roniantoniius/Groceries-Product-Recommendation-Using-Market-Basket-Analysis-FP-Growth

To extract data from greenbasket db

```sql
SELECT 
    oi.order_id as Transaction,
    MAX(CASE WHEN product_rank = 1 THEN p.name END) AS Product1,
    MAX(CASE WHEN product_rank = 2 THEN p.name END) AS Product2,
    MAX(CASE WHEN product_rank = 3 THEN p.name END) AS Product3,
    MAX(CASE WHEN product_rank = 4 THEN p.name END) AS Product4,
    MAX(CASE WHEN product_rank = 5 THEN p.name END) AS Product5,
    MAX(CASE WHEN product_rank = 6 THEN p.name END) AS Product6,
    MAX(CASE WHEN product_rank = 7 THEN p.name END) AS Product7,
    MAX(CASE WHEN product_rank = 8 THEN p.name END) AS Product8,
    MAX(CASE WHEN product_rank = 9 THEN p.name END) AS Product9,
    MAX(CASE WHEN product_rank = 10 THEN p.name END) AS Product10,
    MAX(CASE WHEN product_rank = 11 THEN p.name END) AS Product11,
    MAX(CASE WHEN product_rank = 12 THEN p.name END) AS Product12,
    MAX(CASE WHEN product_rank = 13 THEN p.name END) AS Product13,
    MAX(CASE WHEN product_rank = 14 THEN p.name END) AS Product14,
    MAX(CASE WHEN product_rank = 15 THEN p.name END) AS Product15,
    MAX(CASE WHEN product_rank = 16 THEN p.name END) AS Product16,
    MAX(CASE WHEN product_rank = 17 THEN p.name END) AS Product17,
    MAX(CASE WHEN product_rank = 18 THEN p.name END) AS Product18,
    MAX(CASE WHEN product_rank = 19 THEN p.name END) AS Product19,
    MAX(CASE WHEN product_rank = 20 THEN p.name END) AS Product20,
    MAX(CASE WHEN product_rank = 21 THEN p.name END) AS Product21,
    MAX(CASE WHEN product_rank = 22 THEN p.name END) AS Product22
FROM (
    SELECT 
        oi.order_id,
        oi.product_id,
        ROW_NUMBER() OVER (PARTITION BY oi.order_id ORDER BY oi.product_id) AS product_rank
    FROM order_items oi
) oi
JOIN products p ON p.id = oi.product_id
GROUP BY oi.order_id;
```

#### Imports Required Libraries

In [25]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

#### Load dataset and print two rows

In [26]:
df = pd.read_csv("greenbasket.csv")
df.head()

,Transaction,Product1,Product2,Product3,Product4,Product5,Product6,Product7,Product8,Product9,...,Product13,Product14,Product15,Product16,Product17,Product18,Product19,Product20,Product21,Product22
0,1,Bedside Table African Cherry,Chicken Meat,Green Chili Pepper,Ice Cream,Rice,Soft Drinks,Strawberry,Mug Tree Stand,Plate,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Annibale Colombo Bed,Bedside Table African Cherry,Knoll Saarinen Executive Conference Chair,Chicken Meat,Potatoes,Strawberry,House Showpiece Plant,Carbon Steel Wok,Silver Pot With Glass Cap,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Calvin Klein CK One,Chanel Coco Noir Eau De,Beef Steak,Cat Food,Cooking Oil,Eggs,Fish Steak,Green Bell Pepper,Ice Cream,...,Silver Pot With Glass Cap,Spice Rack,Vaseline Men Body and Face Lotion,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Calvin Klein CK One,Chanel Coco Noir Eau De,Annibale Colombo Sofa,Beef Steak,Cat Food,Cooking Oil,Eggs,Fish Steak,Green Chili Pepper,...,Nescafe Coffee,Fine Mesh Strainer,Hand Blender,Ice Cube Tray,Yellow Peeler,Olay Ultra Moisture Shea Butter Body Wash,Black Sun Glasses,NaN,NaN,NaN
4,5,Calvin Klein CK One,Chanel Coco Noir Eau De,Knoll Saarinen Executive Conference Chair,Beef Steak,Cat Food,Cooking Oil,Eggs,Honey Jar,Potatoes,...,Olay Ultra Moisture Shea Butter Body Wash,Classic Sun Glasses,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df.shape

(7800, 23)

#### Print column names with total empty/NaN value counts

In [28]:
print("Column Name \t Total Null")
df.isnull().sum()

Column Name 	 Total Null


Transaction       0
Product1          0
Product2          0
Product3          0
Product4          0
Product5          0
Product6          0
Product7          0
Product8          0
Product9          0
Product10         3
Product11      1008
Product12      2056
Product13      3087
Product14      4107
Product15      5141
Product16      6197
Product17      6297
Product18      6427
Product19      6562
Product20      6711
Product21      6855
Product22      6994
dtype: int64

#### Replaces 'NaN' values with empty string

In [29]:
df.fillna("", inplace=True)
df.head()

,Transaction,Product1,Product2,Product3,Product4,Product5,Product6,Product7,Product8,Product9,...,Product13,Product14,Product15,Product16,Product17,Product18,Product19,Product20,Product21,Product22
0,1,Bedside Table African Cherry,Chicken Meat,Green Chili Pepper,Ice Cream,Rice,Soft Drinks,Strawberry,Mug Tree Stand,Plate,...,,,,,,,,,,
1,2,Annibale Colombo Bed,Bedside Table African Cherry,Knoll Saarinen Executive Conference Chair,Chicken Meat,Potatoes,Strawberry,House Showpiece Plant,Carbon Steel Wok,Silver Pot With Glass Cap,...,,,,,,,,,,
2,3,Calvin Klein CK One,Chanel Coco Noir Eau De,Beef Steak,Cat Food,Cooking Oil,Eggs,Fish Steak,Green Bell Pepper,Ice Cream,...,Silver Pot With Glass Cap,Spice Rack,Vaseline Men Body and Face Lotion,,,,,,,
3,4,Calvin Klein CK One,Chanel Coco Noir Eau De,Annibale Colombo Sofa,Beef Steak,Cat Food,Cooking Oil,Eggs,Fish Steak,Green Chili Pepper,...,Nescafe Coffee,Fine Mesh Strainer,Hand Blender,Ice Cube Tray,Yellow Peeler,Olay Ultra Moisture Shea Butter Body Wash,Black Sun Glasses,,,
4,5,Calvin Klein CK One,Chanel Coco Noir Eau De,Knoll Saarinen Executive Conference Chair,Beef Steak,Cat Food,Cooking Oil,Eggs,Honey Jar,Potatoes,...,Olay Ultra Moisture Shea Butter Body Wash,Classic Sun Glasses,,,,,,,,


In [30]:
df = df.drop(columns="Transaction", axis=1)

In [31]:
for i in range(df.shape[1]):
    df[f"Product{i + 1}"] = df[f"Product{i + 1}"].apply(lambda word: word.strip())

#### Encode the dataset into true/false values using tansaction encoder

In [32]:
te = TransactionEncoder()
encoded_array  = te.fit_transform(df.values.tolist())
encoded_array

array([[ True, False, False, ..., False, False, False],
       [ True,  True, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       ...,
       [ True, False, False, ...,  True, False, False],
       [ True, False, False, ..., False, False,  True],
       [ True,  True, False, ..., False, False, False]])

#### print encoded array column names

In [33]:
len(te.columns_)

81


#### Convert the encoded array into pandas dataframe

In [34]:
transaction_df = pd.DataFrame(encoded_array, columns= te.columns_)
transaction_df.head()

,,Annibale Colombo Bed,Annibale Colombo Sofa,Apple,Attitude Super Leaves Hand Soap,Bamboo Spatula,Bedside Table African Cherry,Beef Steak,Black Aluminium Cup,Black Sun Glasses,...,Strawberry,Sunglasses,Table Lamp,Tissue Paper Box,Tray,Vaseline Men Body and Face Lotion,Water,Wooden Bathroom Sink With Mirror,Wooden Rolling Pin,Yellow Peeler
0,True,False,False,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,False,False
1,True,True,False,False,False,False,True,False,False,False,...,True,False,False,False,True,False,False,False,False,False
2,True,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
3,True,False,True,False,False,False,False,True,False,True,...,False,False,False,False,False,False,False,False,False,True
4,True,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False


#### Extract frequent items by using apriori algorithm

In [35]:
frequent_items = apriori(transaction_df, min_support=0.01, use_colnames=True)
frequent_items

,support,itemsets
0,0.896667,()
1,0.175128,(Annibale Colombo Bed)
2,0.174615,(Annibale Colombo Sofa)
3,0.176538,(Apple)
4,0.165513,(Attitude Super Leaves Hand Soap)
...,...,...
6476,0.024487,"(, Water, Wooden Rolling Pin)"
6477,0.022564,"(, Yellow Peeler, Water)"
6478,0.023846,"(, Wooden Bathroom Sink With Mirror, Wooden Ro..."
6479,0.022179,"(, Wooden Bathroom Sink With Mirror, Yellow Pe..."


#### Genreate association rules for frequent items

In [36]:
rules = association_rules(frequent_items, min_threshold=0.01)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(),(Annibale Colombo Bed),0.896667,0.175128,0.143846,0.160423,0.916033,1.0,-0.013185,0.982485,-0.470077,0.155015,-0.017827,0.490900
1,(Annibale Colombo Bed),(),0.175128,0.896667,0.143846,0.821376,0.916033,1.0,-0.013185,0.578497,-0.100011,0.155015,-0.728617,0.490900
2,(),(Annibale Colombo Sofa),0.896667,0.174615,0.145897,0.162711,0.931825,1.0,-0.010674,0.985782,-0.414531,0.157661,-0.014423,0.499123
3,(Annibale Colombo Sofa),(),0.174615,0.896667,0.145897,0.835536,0.931825,1.0,-0.010674,0.628304,-0.081424,0.157661,-0.591587,0.499123
4,(),(Apple),0.896667,0.176538,0.146538,0.163426,0.925723,1.0,-0.011758,0.984326,-0.437089,0.158135,-0.015924,0.496746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25435,"(, Wooden Rolling Pin)",(Yellow Peeler),0.148846,0.163462,0.022949,0.154177,0.943203,1.0,-0.001382,0.989024,-0.066073,0.079309,-0.011098,0.147285
25436,"(Yellow Peeler, Wooden Rolling Pin)",(),0.026154,0.896667,0.022949,0.877451,0.978570,1.0,-0.000503,0.843200,-0.021993,0.025502,-0.185958,0.451522
25437,(),"(Yellow Peeler, Wooden Rolling Pin)",0.896667,0.026154,0.022949,0.025593,0.978570,1.0,-0.000503,0.999425,-0.174870,0.025502,-0.000576,0.451522
25438,(Yellow Peeler),"(, Wooden Rolling Pin)",0.163462,0.148846,0.022949,0.140392,0.943203,1.0,-0.001382,0.990165,-0.067150,0.079309,-0.009932,0.147285


#### Print association rules with necessary columns

In [37]:
rules[["antecedents", "consequents", "support", "confidence"]]

,antecedents,consequents,support,confidence
0,(),(Annibale Colombo Bed),0.143846,0.160423
1,(Annibale Colombo Bed),(),0.143846,0.821376
2,(),(Annibale Colombo Sofa),0.145897,0.162711
3,(Annibale Colombo Sofa),(),0.145897,0.835536
4,(),(Apple),0.146538,0.163426
...,...,...,...,...
25435,"(, Wooden Rolling Pin)",(Yellow Peeler),0.022949,0.154177
25436,"(Yellow Peeler, Wooden Rolling Pin)",(),0.022949,0.877451
25437,(),"(Yellow Peeler, Wooden Rolling Pin)",0.022949,0.025593
25438,(Yellow Peeler),"(, Wooden Rolling Pin)",0.022949,0.140392
